## Skill label prediction with image-based features (text and color with descriptive tags)
## Text recognition only

In [344]:
import pandas as pd 
import random
import numpy as np
import copy
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
kfold=KFold(n_splits=10)

# for LSTM (keras with tf backend)
import gzip
import os
import pickle
import requests
import time
import re
os.environ['KERAS_BACKEND']='cntk'
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.models import Sequential, load_model
from keras import regularizers
from keras.optimizers import SGD, Adam
from keras.initializers import he_normal
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.callbacks import History, CSVLogger
from keras.utils import to_categorical
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
nltk.download('wordnet')
nltk.download('punkt')

MAX_DOC_LEN = 40
VOCAB_SIZE = 3000
EMBEDDING_DIM = 100

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/edithzeng/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/edithzeng/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Preprocessing

In [ ]:
vizwiz_features_train_color = pd.read_csv('azure_features_images/data/vizwiz_train_color_recognition.csv',
                                    delimiter=';', engine='python', 
                                    dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'tags':list, 'dominant_colors':list},
                                    quotechar='"', error_bad_lines=False)
vizwiz_features_train_text = pd.read_csv('azure_features_images/data/vizwiz_train_text_recognition.csv',
                                    delimiter=';', engine='python', 
                                    dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'ocr_text':list, 'handwritten_text':list},
                                    quotechar='"', error_bad_lines=False)


vizwiz_features_val_color = pd.read_csv('azure_features_images/data/vizwiz_val_color_recognition.csv',
                                  delimiter=';', engine='python',
                                  dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'tags':list, 'dominant_colors':list},
                                  quotechar='"', error_bad_lines=False)
vizwiz_features_val_text = pd.read_csv('azure_features_images/data/vizwiz_val_text_recognition.csv',
                                    delimiter=';', engine='python', 
                                    dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'ocr_text':list, 'handwritten_text':list},
                                    quotechar='"', error_bad_lines=False)


vqa_features_train_color = pd.read_csv('azure_features_images/data/vqa_train_color_recognition.csv',
                                 delimiter=';', engine='python', 
                                 dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'tags':list, 'dominant_colors':list},
                                 quotechar='"', error_bad_lines=False)
vqa_features_train_text = pd.read_csv('azure_features_images/data/vqa_train_text_recognition.csv',
                                    delimiter=';', engine='python', 
                                    dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'ocr_text':list, 'handwritten_text':list},
                                    quotechar='"', error_bad_lines=False)
vqa_features_val_color = pd.read_csv('azure_features_images/data/vqa_val_color_recognition.csv',
                               delimiter=';', engine='python',
                               dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'tags':list, 'dominant_colors':list},
                               quotechar='"', error_bad_lines=False)
vqa_features_val_text = pd.read_csv('azure_features_images/data/vqa_val_text_recognition.csv',
                                    delimiter=';', engine='python', 
                                    dtype={'qid':str, 'question':str, 'descriptions':list,
                                          'ocr_text':list, 'handwritten_text':list},
                                    quotechar='"', error_bad_lines=False)


vizwiz_targets_train = pd.read_csv('../vizwiz_skill_typ_train.csv',
                                   delimiter=',', quotechar='"',
                                   engine='python', error_bad_lines=False)
vizwiz_targets_val = pd.read_csv('../vizwiz_skill_typ_val.csv',
                                 delimiter=',', quotechar='"', engine='python', error_bad_lines=False)
vqa_targets_train = pd.read_csv('../vqa_skill_typ_train.csv',
                               engine='python', quotechar='"', error_bad_lines=False)
vqa_targets_val = pd.read_csv('../vqa_skill_typ_val.csv',
                               engine='python', quotechar='"', error_bad_lines=False)

In [241]:
vizwiz_features_train_text.head(2)

,qid,question,ocr_text,handwritten_text
0,VizWiz_train_000000000000.jpg,What's the name of this product?,"['b', 'sil', 'leaves', '0.62', 'oz', '(170)']",['NET WT O. 62 02 ( 179)']
1,VizWiz_train_000000000001.jpg,Can you tell me what is in this can please?,[],['^TAKE Three 1^']


In [242]:
vizwiz_train.sample(1)

,IMG,QSN,TXT,OBJ,COL,CNT,OTH,question,ocr_text,handwritten_text,descriptions,tags,dominant_colors
QID,,,,,,,,,,,,,
VizWiz_train_000000009759.jpg,VizWiz_train_000000009759.jpg,what's this?,0,1,0,0,0,what's this?,[],[],"[{'text': 'a close up of a computer', 'confide...","['sitting', 'man', 'computer', 'black', 'stand...","['Grey', 'White']"


In [290]:
def join_feature_target(feature_df_text, feature_df_color, target_df):
    feature_text = copy.deepcopy(feature_df_text)
    feature_color = copy.deepcopy(feature_df_color)
    target = copy.deepcopy(target_df)
    # text features 
    feature_text.rename({'qid': 'QID'}, axis=1, inplace=True)
    feature_text.set_index('QID', inplace=True)
    # color features
    feature_color.rename({'qid': 'QID'}, axis=1, inplace=True)
    feature_color.set_index('QID', inplace=True)
    # join features
    features = feature_text.join(feature_color[['descriptions','tags','dominant_colors']],
                                 on='QID',
                                 how='outer')
    # join features with target
    target = target[['QID', 'IMG', 'QSN', 'TXT', 'OBJ', 'COL', 'CNT', 'OTH']]
    target.set_index('QID', inplace=True)
    target = target.astype(dtype=str)
    df = target.join(features, on='QID', how='inner')
    df['descriptions'].astype(list)
    return df

def lem(s):
    arr = s.split(" ")
    lem = WordNetLemmatizer()
    op = ""
    for w in arr:
        word = lem.lemmatize(w) + ' '
        op += word
    return op

def preprocess_text(feature_columns):
    """ output an nparray with single document per data point """
    ip = copy.deepcopy(feature_columns).values
    op = []
    for i in range(ip.shape[0]):
        doc      =  ""
        for j in range(ip.shape[1]):
            # clean up chars
            s    =  str(ip[i][j])
            s    =  s.translate({ord(c): "" for c in "!@#$%^&*()[]{};:,./<>?\|`~-=_+'"}).lower() + " "
            if j == 1:             # clean descriptions
                s = re.sub(r'confidence\s+\d+', '', s)
                s = re.sub(r'text', '', s)
            # lexicon normalize
            s    = lem(s)
            doc  += s
        op.append(doc)
    op = np.asarray(op)
    return op

In [291]:
vizwiz_train   = join_feature_target(vizwiz_features_train_text, 
                                   vizwiz_features_train_color, 
                                   vizwiz_targets_train)
vizwiz_val     = join_feature_target(vizwiz_features_val_text, 
                                 vizwiz_features_val_color, 
                                 vizwiz_targets_val)

In [326]:
# create X and Y
features_train = preprocess_text(vizwiz_train[['QSN', 
                                              'descriptions', 'tags', 'dominant_colors', 
                                              'handwritten_text', 'ocr_text']])
txt_train      = vizwiz_train["TXT"].values
obj_train      = vizwiz_train["OBJ"].values
col_train      = vizwiz_train["COL"].values
cnt_train      = vizwiz_train["CNT"].values

features_val   = preprocess_text(vizwiz_val[['QSN', 
                                             'descriptions', 'tags', 'dominant_colors',
                                             'handwritten_text', 'ocr_text']])
txt_val        = vizwiz_val["TXT"].values.astype('float32')
obj_val        = vizwiz_val["OBJ"].values
col_val        = vizwiz_val["COL"].values.astype('float32')
cnt_val        = vizwiz_val["CNT"].values

In [297]:
features_train[random.randint(0,len(features_val))]

'is it possible to tell the ingredient of this pizza and the brand  thanks   a pizza sitting on top of a table   indoor table sitting food pizza plate red phone eating cat laying  brown red  stone tombsto original    '

In [108]:
features_val[random.randint(0,len(features_val))]

'what is this medicine text a close up of a bottle  indoor sitting bottle green table small food black counter white lotion fruit laying phone brush grey active ingredient  new cetirizine hot 10 ndc 30142  458  95 uses temporary relievesto  runny nose  sneezing 1 warnings do not use in 24 hour ingredients or to an artnozz have liver or kidney deese wey all day doctor or pharmacistbe this product " drowsiness allergy tranquilizers may not machinery stop use and medical help right away  cetirizine hydrochloride tablets  recommended  i prays 10 ing  antihistamine children  in case downdue directions adults and 45 tablets active i gredie•ltp" hoc 3014245895 24 hour allday allergy ceirizåne hydrochloride tablets 10 mgantihistamine 45 tablets '

In [298]:
# tokenize
tok        = Tokenizer(num_words=VOCAB_SIZE, 
                       filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                       lower=True,
                       split=" ")
tok.fit_on_texts(features_train)

# create sequences & pad
train_seq  = tok.texts_to_sequences(features_train)
train_seq  = sequence.pad_sequences(train_seq, maxlen=MAX_DOC_LEN)
val_seq    = tok.texts_to_sequences(features_val)
val_seq    = sequence.pad_sequences(val_seq, maxlen=MAX_DOC_LEN)

In [299]:
# check class distribution
text_recognition_labels = to_categorical(np.asarray(txt_train)).astype('float32')
color_recognition_labels = to_categorical(np.asarray(col_train)).astype('float32')
print('Number of samples each class - Vizwiz - train')
print('Text recognition', text_recognition_labels.sum(axis=0))
print('Color recognition', color_recognition_labels.sum(axis=0))
n_classes = 2

Number of samples each class - Vizwiz - train
Text recognition [6247. 8010.]
Color recognition [8844. 5413.]


## LSTM model (skip-gram word2vec)
config for cpu only

In [255]:
import numpy as np
import pandas as pd
import os
import pickle
import requests
import time
import gzip

os.environ['KERAS_BACKEND'] = 'cntk'

from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.models import Sequential, load_model
from keras import regularizers
from keras.optimizers import SGD, 
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.callbacks import History, CSVLogger
from keras.utils import to_categorical

import nltk 
import gensim
import logging

from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

MAX_DOC_LEN = 40
VOCAB_SIZE = 3000
EMBEDDING_DIM = 100

#MAX_DOC_LEN = 40
#VOCAB_SIZE = 7000
#EMBEDDING_DIM = 150

In [300]:
# punkt sentence level tokenizer
sent_lst = []
for doc in features_train:
    sentences = nltk.tokenize.sent_tokenize(doc)
    for sent in sentences:
        word_lst = [w for w in nltk.tokenize.word_tokenize(sent) if w.isalnum()]
        sent_lst.append(word_lst)

In [301]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
word2vec_model = gensim.models.Word2Vec(sentences=sent_lst,
                                        min_count=6,
                                        size=EMBEDDING_DIM,
                                        sg=1,
                                        workers=os.cpu_count())

2019-02-26 22:45:30,009 : INFO : collecting all words and their counts
2019-02-26 22:45:30,009 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-02-26 22:45:30,072 : INFO : PROGRESS: at sentence #10000, processed 383294 words, keeping 24880 word types
2019-02-26 22:45:30,099 : INFO : collected 31762 word types from a corpus of 541792 raw words and 14257 sentences
2019-02-26 22:45:30,100 : INFO : Loading a fresh vocabulary
2019-02-26 22:45:30,126 : INFO : effective_min_count=6 retains 3565 unique words (11% of original 31762, drops 28197)
2019-02-26 22:45:30,126 : INFO : effective_min_count=6 leaves 501856 word corpus (92% of original 541792, drops 39936)
2019-02-26 22:45:30,135 : INFO : deleting the raw counts dictionary of 31762 items
2019-02-26 22:45:30,136 : INFO : sample=0.001 downsamples 67 most-common words
2019-02-26 22:45:30,136 : INFO : downsampling leaves estimated 318802 word corpus (63.5% of prior 501856)
2019-02-26 22:45:30,143 : INFO : estima

In [302]:
embeddings_index = {}

for word in word2vec_model.wv.vocab:
    coefs = np.asarray(word2vec_model.wv[word], dtype='float32')
    embeddings_index[word] = coefs

print('Total %s word vectors' % len(embeddings_index))

# Initial word embedding
embedding_matrix = np.zeros((VOCAB_SIZE, EMBEDDING_DIM))

for word, i in tok.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None and i < VOCAB_SIZE:
        embedding_matrix[i] = embedding_vector

Total 3565 word vectors


In [369]:
def lstm_create_train(labels, learning_rate, lstm_dim, batch_size, num_epochs, optimizer_param, regularization=1e-7):
    
    l2_reg = regularizers.l2(regularization)
    
    # init model
    initializer = he_normal(seed=42)
    embedding_layer = Embedding(VOCAB_SIZE,
                                EMBEDDING_DIM,
                                input_length=MAX_DOC_LEN,
                                trainable=True,
                                mask_zero=False,
                                embeddings_regularizer=l2_reg,
                                weights=[embedding_matrix])
    lstm_layer = LSTM(units=lstm_dim, kernel_regularizer=l2_reg)
    dense_layer = Dense(n_classes,
                        kernel_initializer=initializer,
                        activation='softmax', 
                        kernel_regularizer=l2_reg)

    model = Sequential()
    model.add(embedding_layer)
    model.add(Bidirectional(lstm_layer))
    model.add(Dropout(0.5))
    model.add(dense_layer)
    
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer_param,
                  metrics=['acc'])
    history = History()
    csv_logger = CSVLogger('./LSTM/text/{}_{}_{}_{}.log'.format(learning_rate, regularization, batch_size, num_epochs),
                           separator=',',
                           append=True)
    t1 = time.time()
    # model fit
    model.fit(train_seq,
              labels.astype('float32'),
              batch_size=batch_size,
              epochs=num_epochs,
              callbacks=[history, csv_logger],
              verbose=2)
    t2 = time.time()
    # save hdf5
    model.save('./LSTM/text/{}_{}_{}_{}_model.h5'.format(learning_rate, regularization, batch_size, num_epochs))
    np.savetxt('./LSTM/text/{}_{}_{}_{}_time.txt'.format(learning_rate, regularization, batch_size, num_epochs), 
               [regularization, (t2-t1) / 3600])
    with open('./LSTM/text/{}_{}_{}_{}_history.txt'.format(learning_rate, regularization, batch_size, num_epochs), "w") as res_file:
        res_file.write(str(history.history))

In [366]:
# dropout 0.2
learning_rate = 0.08
regularization = 1e-8
lstm_create_train(labels=text_recognition_labels,
                  learning_rate=learning_rate,
                  lstm_dim=100, 
                  batch_size=60,
                  num_epochs=30,
                  optimizer_param=SGD(lr=learning_rate, nesterov=True),
                  regularization=regularization)

model = load_model('./LSTM/text/{}_{}_model.h5'.format(learning_rate,regularization))
preds = model.predict(val_seq, verbose=1)
print(("Accuracy = {} \t AUC = {}".format(accuracy_score(txt_val, preds.argmax(axis=1)), 
       roc_auc_score(txt_val, preds[:,1]))))

Epoch 1/30
 - 14s - loss: 0.5174 - acc: 0.7549
Epoch 2/30
 - 14s - loss: 0.4501 - acc: 0.7979
Epoch 3/30
 - 14s - loss: 0.4322 - acc: 0.8040
Epoch 4/30
 - 14s - loss: 0.4159 - acc: 0.8191
Epoch 5/30
 - 14s - loss: 0.4048 - acc: 0.8239
Epoch 6/30
 - 14s - loss: 0.3945 - acc: 0.8284
Epoch 7/30
 - 14s - loss: 0.3878 - acc: 0.8341
Epoch 8/30
 - 14s - loss: 0.3813 - acc: 0.8380
Epoch 9/30
 - 14s - loss: 0.3782 - acc: 0.8398
Epoch 10/30
 - 14s - loss: 0.3692 - acc: 0.8426
Epoch 11/30
 - 16s - loss: 0.3684 - acc: 0.8439
Epoch 12/30
 - 16s - loss: 0.3615 - acc: 0.8463
Epoch 13/30
 - 15s - loss: 0.3558 - acc: 0.8502
Epoch 14/30
 - 15s - loss: 0.3555 - acc: 0.8533
Epoch 15/30
 - 16s - loss: 0.3562 - acc: 0.8508
Epoch 16/30
 - 15s - loss: 0.3526 - acc: 0.8524
Epoch 17/30
 - 15s - loss: 0.3469 - acc: 0.8578
Epoch 18/30
 - 15s - loss: 0.3448 - acc: 0.8568
Epoch 19/30
 - 15s - loss: 0.3454 - acc: 0.8571
Epoch 20/30
 - 15s - loss: 0.3433 - acc: 0.8588
Epoch 21/30
 - 14s - loss: 0.3413 - acc: 0.8608
E

In [ ]:
# dropout = .5
# Adam optimizer
lstm_create_train(labels=text_recognition_labels,
                  lstm_dim=100, 
                  batch_size=50,
                  num_epochs=30, 
                  optimizer_param=Adam(lr=0.0001),
                  regularization=1e-8)

model = load_model('./LSTM/lstm_{0}_model.h5'.format(1e-8))
preds = model.predict(val_seq, verbose=1)
print(("Accuracy = {0} \t AUC = {1}".format(accuracy_score(txt_val, preds.argmax(axis=1)), 
       roc_auc_score(txt_val, preds[:,1]))))

In [ ]:
learning_rate  = [1e-5, 1e-6, 1e-8, 1e-10]
regularization = [1e-4, 1e-8, 1e-10, 0]
batch_size     = [20, 50, 100, 150]
epochs         = [30, 50, 80, 100]

# dropout = 0.5, GSD Nesterov

for L in learning_rate:
    for R in regularization:
        for B in batch_size: 
            for E in epochs:
                lstm_create_train(labels=text_recognition_labels,
                                  learning_rate=L,
                                  lstm_dim=100, 
                                  batch_size=B,
                                  num_epochs=E, 
                                  optimizer_param=SGD(lr=L, nesterov=True),
                                  regularization=R)
                model = load_model('./LSTM/text/{}_{}_{}_{}_model.h5'.format(L,R,B,E))
                preds = model.predict(val_seq, verbose=0)
                print("Learning rate: {} Regularization: {} Batch size: {} Epoch: {}".format(L,R,B,E))
                print(("Accuracy = {0} \t AUC = {1}".format(accuracy_score(txt_val, preds.argmax(axis=1)), 
                                                            roc_auc_score(txt_val, preds[:,1]))))

Epoch 1/30
 - 40s - loss: 1.9788 - acc: 0.5244
Epoch 2/30
 - 39s - loss: 1.9755 - acc: 0.5284
Epoch 3/30
 - 40s - loss: 1.9755 - acc: 0.5301
Epoch 4/30
 - 40s - loss: 1.9719 - acc: 0.5423
Epoch 5/30
 - 40s - loss: 1.9710 - acc: 0.5453
Epoch 6/30
